<a href="https://colab.research.google.com/github/viraatdas/speedchallenge/blob/master/speedchallenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/viraatdas/speedchallenge.git

fatal: destination path 'speedchallenge' already exists and is not an empty directory.


In [4]:
%cd speedchallenge/

/content/speedchallenge


In [3]:
!pip install --upgrade opencv-python

    100% |████████████████████████████████| 25.4MB 1.3MB/s 
imgaug 0.2.8 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.8 which is incompatible.
  Found existing installation: opencv-python 3.4.5.20
    Uninstalling opencv-python-3.4.5.20:
      Successfully uninstalled opencv-python-3.4.5.20


In [6]:

!pwd

/content/speedchallenge


In [0]:
import cv2 as cv
import numpy as np


#Getting speed from the text file
speed = []
location = "/content/speedchallenge/data/train.txt"
with open(location) as f:
    speed = list(f)

speed = [x.strip() for x in speed]
iter_speed = iter(speed) #making the speed list an iterable



In [0]:
# retrieving all frames from video and saving them in all_frames
# all_frames[np.ndarry] = speed for that particular frame
location = "/content/speedchallenge/data/train.mp4"
cap = cv.VideoCapture(location)
all_frames = []
ret, frame1 = cap.read()
i = 0
while i < len(speed):
    temp = []
    temp.append(frame1)
    try:
        temp.append(float(next(iter_speed)))
    except:
        continue
    all_frames.append(temp)
    ret, frame1 = cap.read()
    i+=1


In [0]:
#applying Adaptive Guassian Thresholding to account for illumination changes
i = 0
while i < len(all_frames):
    img = cv.cvtColor(all_frames[i][0], cv.COLOR_BGR2GRAY) #convert to grayscale
    img = cv.medianBlur(img,5) #median blur

    #After applying adaptive gaussian thresholding, saving it to all_frames
    all_frames[i][0] = cv.adaptiveThreshold(img, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, \
                                            cv.THRESH_BINARY, 11, 2)
    i += 1



#since video was analyze in pairs of successive frames
#80% of the intitial frames were training
#and 20% were used for validation
eighty_percent = int(0.8*len(all_frames))
training, validation = all_frames[:eighty_percent], all_frames[eighty_percent:]



#Dense Optical Flow based on the Gunner Farneback's algorithm
cap = cv.VideoCapture(location)
ret, frame1 = cap.read()
prvs = cv.cvtColor(frame1,cv.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame1)
prvs = cv.cvtColor(frame1,cv.COLOR_BGR2GRAY)
hsv[...,1] = 255

i = 1
while i < len(training):
    next = training[i][0]

    flow = cv.calcOpticalFlowFarneback(prvs,next, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    mag, ang = cv.cartToPolar(flow[...,0], flow[...,1])
    hsv[...,0] = ang*180/np.pi/2
    hsv[...,2] = cv.normalize(mag,None,0,255,cv.NORM_MINMAX)
    bgr = cv.cvtColor(hsv,cv.COLOR_HSV2BGR)
    cv.imshow('frame2',bgr)
    k = cv.waitKey(30) & 0xff
    if k == 27:
        break
    elif k == ord('s'):
        cv.imwrite('opticalfb.png',training[i][0])
        cv.imwrite('opticalhsv.png',bgr)
    prvs = next
    i += 1

cap.release()
cv.destroyAllWindows()